In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/modified-mudras/mudra_word_mapping.json


In [3]:
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the JSON file
with open("/kaggle/input/modified-mudras/mudra_word_mapping.json", "r", encoding="utf-8") as f:
    mudra_dict = json.load(f)

# Search function
def find_mudras_by_query(query):
    query = query.strip().lower()
    results = []

    for mudra, entries in mudra_dict.items():
        for entry in entries:
            word_match = query in entry["word"].lower()
            trans_match = any(query in t.lower() for t in entry["translations"])
            if word_match or trans_match:
                results.append({
                    "mudra": mudra,
                    "word": entry["word"],
                    "translations": entry["translations"]
                })
    return results

# Widgets

title = widgets.HTML(
    value="<h2 style='color:#3b5998; font-family:Arial;'>Mudra Finder 🔍</h2>"
)

text_input = widgets.Text(
    value='',
    placeholder='Enter word or meaning (e.g., wind, vayo)',
    description='🔎 Search:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', padding='8px 0px 8px 10px')
)

search_button = widgets.Button(
    description="Search",
    button_style='info',
    tooltip="Click to search",
    layout=widgets.Layout(width='20%', margin='10px 10px 10px 10px')
)

output = widgets.Output()

# Simulated "Enter" behavior
def run_query(query):
    with output:
        clear_output()
        results = find_mudras_by_query(query)
        if results:
            display(widgets.HTML(f"<h4>🔍 Results for: <i>{query}</i></h4>"))
            for res in results:
                display(widgets.HTML(
                    f"<b>👉 Mudra:</b> {res['mudra']}<br>"
                    f"<b>Word:</b> {res['word']}<br>"
                    f"<b>Meanings:</b> {', '.join(res['translations'])}<br><br>"
                ))
        else:
            display(widgets.HTML(
                f"<span style='color:red;'>❌ No mudras found for: <i>{query}</i></span>"
            ))

def on_button_click(b):
    query = text_input.value.strip()
    if query:
        run_query(query)

# Attach events
search_button.on_click(on_button_click)

# --- Layout and Display ---
ui = widgets.VBox([
    title,
    widgets.HBox([text_input, search_button]),
    output
])

display(ui)

In [4]:
import json
import re
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the mudra dictionary
with open("/kaggle/input/modified-mudras/mudra_word_mapping.json", "r", encoding="utf-8") as f:
    mudra_dict = json.load(f)

title = widgets.HTML(value="<h2 style='color:#8B0000; font-family:Georgia;'>Shloka to Mudra Mapper 🙏</h2>")
# Widget for shloka input
shloka_input = widgets.Textarea(
    value='',
    placeholder='Paste your shloka or verse here...',
    description='Shloka:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': 'initial'}
)

search_button = widgets.Button(
   description='Find Mudras',button_style='success',icon='search',layout=widgets.Layout(width='150px', margin='10px 0px 10px 0px')
)

output = widgets.Output()

# Mudra matching function
def get_mudras_for_shloka(shloka_text):
    words = re.findall(r'\b\w+\b', shloka_text.lower())
    matched_mudras = {}

    for word in words:
        for mudra, entries in mudra_dict.items():
            for entry in entries:
                if word in entry["word"].lower() or any(word in t.lower() for t in entry["translations"]):
                    if mudra not in matched_mudras:
                        matched_mudras[mudra] = []
                    matched_mudras[mudra].append({
                        "word": entry["word"],
                        "translation": entry["translations"]
                    })

    with output:
        clear_output()
        if matched_mudras:
            print("🪷 Mudras matching this shloka/concept:\n")
            for mudra, matches in matched_mudras.items():
                print(f"🖐️ Mudra: {mudra}")
                for m in matches:
                    print(f"   ↳ Word: {m['word']}, Meanings: {', '.join(m['translation'])}")
                print()
        else:
            print("❌ No mudras found for this shloka.")

# Button click handler
def on_search_clicked(b):
    get_mudras_for_shloka(shloka_input.value)

# Bind event
search_button.on_click(on_search_clicked)

# Display UI
display(widgets.VBox([title,shloka_input, search_button, output]))
